In [ ]:
#make IPython notebook matplotlib plot inline
%matplotlib inline 

#import modules and libraries
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from pathlib import Path
from PIL import Image
from scipy import ndimage
from skimage import measure
from skimage.io import imread,imsave

INPUT_DIR = '/work/data/jilehtim/HARRI/PYTHON_INPUT'
OUTPUT_DIR = '/work/data/jilehtim/HARRI/PYTHON_OUTPUT'

INPUT_DIR = Path(INPUT_DIR)
if not INPUT_DIR.is_dir():
    raise ValueError('Please check INPUT_DIR.')

OUTPUT_DIR = Path(OUTPUT_DIR)
if not OUTPUT_DIR.is_dir():
    raise ValueError('Please check OUTPUT_DIR.')

MIN_FRAMES = 4
MIN_HEXAGONS_IN_2D = 3
PIXELS_PER_HEXAGON = 150


In [ ]:
def exclude_small_regions_in_2d(im):
    blobs = im == 255
    labels = measure.label(blobs)
    if labels.max() == 0:
        return im

    #props = measure.regionprops_table(labels, properties=('label','area',))
    #df = pd.DataFrame(props)
    #print(df)

    regions = measure.regionprops(labels)
    for props in regions:
        if props.area < MIN_HEXAGONS_IN_2D * PIXELS_PER_HEXAGON:
            labels[labels == props.label] = 0  
            
    return (labels > 0) * 255

def exclude_short_blinks_in_3d(im):
    # select white pixels into blobs
    blobs = im == 255
    # label blobs
    labels = measure.label(blobs)
    # measure labeled blobs
    props = measure.regionprops_table(labels)
    # read measurements into a dataframe
    df = pd.DataFrame(props)
    # add column for and calculate blink duration
    df['duration'] = df['bbox-3'] - df['bbox-0']

    print(df.shape)
    print(df.head())

    # filter out short blinks
    df = df[df['duration'] >= MIN_FRAMES]
    print(df.shape)
    long_blinks = df.label.values
    print(long_blinks)

    # set label to zero in labels image if the label is not in long_blinks
    ul = np.unique(labels)
    for l in ul:
        if l not in long_blinks:
            labels[labels==l] = 0

    from skimage import img_as_uint
    labels = img_as_uint(labels)
    
    return labels, df
    

In [ ]:
# process all .ome.tif files in input directory    
files = INPUT_DIR.glob('*.ome.tif')
im = None
im2 = None
labels = None
for f in files:
    print(f)
    im = imread(str(f), plugin="tifffile")
    im2 = im.copy()
    print(im.shape)
    print(im.dtype)
    
    for i in range(0, im.shape[0]):
        im2[i] = exclude_small_regions_in_2d(im[i])
    print(im2.shape)
    print(im2.dtype)
                   
    labels, df = exclude_short_blinks_in_3d(im2)
    
    # save modified labels image
    output = OUTPUT_DIR  / (f.stem + '_long_blinks.ome.tif')
    print(str(output))
    imsave(str(output),labels)

    # save measurements
    output = OUTPUT_DIR / (f.stem + '_long_blinks.csv')
    print(str(output))
    df.to_csv(output, index=False)


In [ ]:
z = 47
z = 68
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
axs[0].imshow(im[z],cmap='coolwarm')
axs[1].imshow(im2[z],cmap='coolwarm')

In [ ]:
z = 46
labels = exclude_small_regions_in_2d(im[z])

fig, axs = plt.subplots(1, 2, figsize=(15, 10))
axs[0].imshow(im[z],cmap='coolwarm')
axs[1].imshow(labels,cmap='coolwarm')


In [ ]:
help(imsave)

In [ ]:
print(im.max())
print(im2.max())